In [2]:
# IMPORTING REQUIRED LIBRARIES
import cv2
import numpy as np
import serial

In [3]:
def direction(Cx,Cy,frame):
    global rotate
    
    FB=0 # 0-STAY,1-FORWARD,2-BACKWARD
    LR=0 # 0-STAY,1-LEFT,2-RIGHT
    data='0'
    
    # CONDITION FOR MOVING FORWARD
    if(Cx>=170 and Cx<=470 and Cy<270):
        FB=1
    else:
        FB=0
        
    # CONDITION FOR MOVING LEFT AND RIGHT
    if(Cx<=170):
        LR=2
    elif(Cx>=470):
        LR=1
    else:
        LR=0
        
    # 0-STAY
    # 1-FORWARD
    # 2-LEFT
    # 3-RIGHT
    
    if(FB==0 and LR==0):
        data='0'
        frame=cv2.putText(frame,"STAY",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
        
    if(FB==1):
        data='1'
        frame=cv2.putText(frame,"FORWARD",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
        
    if(LR==1):
        data='2'
        frame=cv2.putText(frame,"LEFT",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
        rotate="LEFT"
        
    if(LR==2):
        data='3'
        frame=cv2.putText(frame,"RIGHT",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
        rotate="RIGHT"
        
    #ser.write(data.encode())  
        

In [ ]:
#ser=serial.Serial("COM7",9600,timeout=1)
cap=cv2.VideoCapture(1)
rotate="STAY" # THIS IS USED TO ROTATE THE BOT IN THE LAST DETECTED DIRECTION IF BALL IS OUT OF RANGE OF CAMERA 
data='0'

while(1):
    ret,frame=cap.read()
    frame=cv2.flip(frame,1)
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV) # CHANGING TO HSV COLOUR SPACE SO THAT BALL CAN BE DETECTED
    
    # DRAWING THE DEADZONE
    frame=cv2.line(frame,(170,0),(170,480),(0,0,255),3)
    frame=cv2.line(frame,(470,0),(470,480),(0,0,255),3)
    frame=cv2.line(frame,(170,290),(470,290),(0,0,255),3)
    
    # LOWER AND UPPER HSV RANGE OF PINK COLOUR
    lower=np.array([135,17,255])
    upper = np.array([187,159,255])
    
    # CONVERTING TO BINARY IMAGE 
    th=cv2.inRange(hsv,lower,upper)
    
    # FINDING THE CONTOURS
    contours,_=cv2.findContours(th,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    # IF CONTOURS ARE FOUND
    if(len(contours)!=0):
        
        # FINDING THE MAX CONTOUR
        max_cnt=max(contours,key=cv2.contourArea)
        
        # FINDING THE AREA TO REMOVE SMALL NOISES
        area=cv2.contourArea(max_cnt)
        
        if(area>200):
            x,y,w,h=cv2.boundingRect(max_cnt)
            
            # CENTER COORDINATES OF BALL
            Cx=x+w//2
            Cy=y+h//2
            
            # DRAWING BOX AROUND BALL
            frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
            
            # DRAWING CENTER OF BALL
            frame=cv2.circle(frame,(Cx,Cy),5,(0,0,255),-1)
            
            # FUNCTION TO GIVE THE DIRECTION TO MOVE TOWARDS BALL
            direction(Cx,Cy,frame)
            
        else:
            if(rotate=="STAY"):
                data='0'
                frame=cv2.putText(frame,"STAY",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
                
            if(rotate=='LEFT'):
                data='2'
                frame=cv2.putText(frame,"LEFT",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
                
            if(rotate=='RIGHT'):
                data='3'
                frame=cv2.putText(frame,"RIGHT",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
                
            #ser.write(data.encode())   
            
    else:
        if(rotate=="STAY"):
            data='0'
            frame=cv2.putText(frame,"STAY",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
                
        if(rotate=='LEFT'):
            data='2'
            frame=cv2.putText(frame,"LEFT",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
                
        if(rotate=='RIGHT'):
            data='3'
            frame=cv2.putText(frame,"RIGHT",(10,100),cv2.FONT_HERSHEY_COMPLEX,3,(0,255,0),2,cv2.LINE_AA)
                
        #ser.write(data.encode())
    
    
    cv2.imshow("WIN",frame)
    if(cv2.waitKey(1)==ord('q')):
        break
cap.release()
cv2.destroyAllWindows()